### Install all required packages for running the notebook

In [ ]:
%pip install qiskit==0.42.1
%pip install amazon-braket-sdk==1.37.1
%pip install tweedledum==1.1.1
%pip install qiskit-ibm-runtime==0.9.3
%pip install pylatexenc==2.10

### Load required functions and libraries

In [ ]:
from functionsForProtocol import *
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
import matplotlib.pyplot as plt

### Loading Boolean network rules to build the circuit

In [ ]:
rulestxt = "./networks/Giacomantonio2010.txt"

This function call parses the Boolean rules provided in the rulestxt file into quantum circuits, creating a new *networkname*_rulefile.py file in the process.
A single circuit for performing 'Tmax' consecutive synchronous state transitions is created from this.
The circuit is initialized with Hadamard gates, meaning the system starts in a uniform superposition of all $2^n$ states.
A total of 'nrshots' shots are performed on a statevector simulator.
Shots indicate repeated initializations, runs, and measurements of the circuit, thus building up a probability distribution of measurement outcomes.
The *quantumStateTransition()* function can take on further arguments which will be used for other types of dynamic analyses, as will be explained in the corresponding cells below.

In [ ]:
#Visualizing the quantum circuit for performing a single state transition
#returnCircuitOnly=True will return only the quantum circuit object from quantumStateTransition() without performing any measurements
transitionCircuitVisualization = quantumStateTransition(rulestxt, Tmax=1, returnCircuitOnly=True)
transitionCircuitVisualization.draw(output='mpl')

### Simulate repeated quantum state transitions, convergence to attractor superposition

In [ ]:
CorticalNet4Transitions = quantumStateTransition(rulestxt, Tmax=4, nrshots=1000)
print(CorticalNet4Transitions)

This function call returned the network's classical attractor distribution of the states 10010 (classically 87.5%) and 01101 (classically 12.5%).
Results may vary slightly due to the randomness of measurements.
After 4 transitions, each of the 32 possible states will have reached an attractor. This transient time will differ for each particular network and generally scales linearly with the number of network components n for biological networks (see Aldana, Physica D 185 (2003), Boolean dynamics of networks with scale-free topology).

### Use mock backends to mimick the noise profiles of real QPUs
As before, the quantumStateTransition() function call parses the Boolean rules into a quantum circuit performing Tmax consecutive state transitions.
However, here the 'backend' argument is used to specify an 'fake' mock backend whose noise characteristics are applied to the statevector simulator.
The circuit is transpiled to the set of gates available on the specified backend. The argument optimization_level aims to optimize this transpilation process and takes values from 0 to 3, indicating increasing amounts of circuit optimization.
The use of such fake backends may give a more realistic estimate of the expected accuracy of a quantum circuit when compared to a noiseless simulator.
It also does not require access to a physical QPU. It thus saves the associated cost of sending tasks to a real device.

In [ ]:
from qiskit.providers.fake_provider import FakeToronto
backend = FakeToronto()
CorticalNet_noisySimulatorTransition = quantumStateTransition(rulestxt, Tmax=1, nrshots=1000, backend=backend,
                                                              optimization_level=3, returnCircuitDepth=True)
print(CorticalNet_noisySimulatorTransition)

#Calculate fidelity as implemented in qiskit, compare single noisy to single exact transition
from qiskit.quantum_info import hellinger_fidelity
CorticalNet_ExactTransition = quantumStateTransition(rulestxt, Tmax=1, nrshots=1000)
print("Fidelity of noisy transition relative to exact simulator:")
print(hellinger_fidelity(CorticalNet_ExactTransition, CorticalNet_noisySimulatorTransition))

#Implementation of normalized fidelity measure as described in Weidner et al.
#Overlap of output that is equivalent to overlap with a uniform (i.e. maximally noisy) distribution is considered as a fidelity of 0.
print("Normalized fidelity measure:")
print(normalized_fidelity(idealDistr=CorticalNet_ExactTransition, outputDistr=CorticalNet_noisySimulatorTransition))

### Visualization of outcomes between noiseless and noisy simulation
Run the cell above to generate CorticalNet_ExactTransition and CorticalNet_noisySimulatorTransition

In [ ]:
#Generate a barplot visualizing the dictionaries containing the measurement probabilities of an exact and noisy simulation of a quantum state transition
barWidth = 0.15
# set heights of bars (y values to plot)
bars0 = probdict2list(CorticalNet_ExactTransition)
bars1 = probdict2list(CorticalNet_noisySimulatorTransition)
# Set position of bar on X axis
r0 = np.arange(len(bars0))
r1 = [x + barWidth for x in r0]
fig, ax = plt.subplots()
fig.set_size_inches(w=10, h=5.5)
ax.bar(r0, bars0, color='black', width=barWidth, edgecolor='white', label='Exact simulation')
ax.bar(r1, bars1, color='red', width=barWidth, edgecolor='white', label='Noisy simulation')
ax.set_ylabel("Probability", fontsize=18)
# Add xticks on the middle of the group bars
ax.set_xlabel('State', fontsize=18)
ax.set_title("Exact and noisy simulation of a quantum state transition", fontsize=15)
ax.set_yticks([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
ax.legend(framealpha=1, loc="upper right", fontsize=15)
ax.grid(color='gray', linestyle='dashed', axis='y', alpha=0.5)
ax.set_xticks(list(range(0,32)))
ax.xaxis.set_ticklabels(["00000", "00001", "00010", "00011", "00100",
               "00101", "00110", "00111", "01000", "01001",
               "01010", "01011", "01100", "01101", "01110",
               "01111", "10000", "10001", "10010", "10011",
               "10100", "10101", "10110", "10111", "11000",
               "11001", "11010", "11011", "11100", "11101",
               "11110", "11111"], rotation=75, ha="center")
plt.ylim(0,0.5)
plt.show()

### Tune activity biases in initial superposition state
In a circuit implementing forward state transitions, the initial Hadamard gates can be replaced with tunable $R_y(\theta)$ gates.
These parameters $\theta$ are specified via the initActivities argument, and can be specified as thetatype "angle" or "radian".
Biasing the initial state of the first gene in the network with $\theta=0°$ leads to the exploration of the subspace of the state transition graph
of size $2^{n-1}$ where this gene is inactive. Values in the range $0° < \theta < 90°$ will explore the full state space,
but return different probabilities than a classical simulation as the initial states now no longer have equal amplitudes of $1/2^n$, but are biased towards inactivation of the modified gene's initial state. Likewise, values of $90° < \theta < 180°$ result in a bias of state amplitudes towards the gene being active.
In this manner, specific subspaces of biologically plausible activity ranges for multiple genes can be explored
which may shift attractor probabilities or leave them unchanged. Thus, the sensitivity or lack of response of the phenotypical landscape towards particular genes activity can be quantified.
This analysis is implementable by modifying only a single layer of gates in the circuit's initialization.

In [ ]:
#Variation in first gene Fgf8
thetavalues = list(range(0,190,10)) #vary angle from 0° to 180°
for theta in thetavalues:
    print("Gate used for initialising the first gene is R_y(theta = " + str(theta) + "°)")
    ThetaVariationGiacomantonio = quantumStateTransition(rulestxt, Tmax=4, nrshots=1000,
                                       initActivities=[theta,90,90,90,90], initPerturbed=None, returnCircuitOnly=False,
                                       thetatype="angle")
    print(ThetaVariationGiacomantonio)

#10010 is the only attractor when Fgf8 is initialised with theta=0 -> Fgf8(t=0) = |0>
#The probability of 01101 increases as Fgf8 initial activity gradually increases
# -> The entire 16-state XXXX0 subspace leads to only one attractor

### Simultaneous analysis of knockouts and overexpressions using superposition perturbations
As before, the initialization of gene activities can be tuned via $R_y(\theta)$ gates.
Additionally, the boolean vector initPertubed specifies if a gene should be perturbed using this initial superposition state.
For those values that are set 'True', the same initial (possibly biased) superposition state created by the $R_y(\theta)$ rotation will be re-used for that gene after every quantum state transition.
This is equivalent to simultaneously performing an overexpression (OE) and a knockout (KO) simulation for classical perturbations of this network component.
Multiple components can be perturbed in this manner. For values $0° < \theta < 90°$ the attractors obtained from a knockout simulation will appear
with a proportionately larger probability, likewise for the attractors obtained from an overexpression perturbation for $90° < \theta < 180°$.
Here, single and double superposition perturbations are performed.
The former returns the attractors from a KO and OE of the third component Pax6. The latter returns the union set of all four possible double perturbations of Pax6 and Coup_tfi (i.e KO+KO, KO+OE, OE+KO, OE+OE) with the attractors weight being determined both by their basin sizes in the perturbed system as well as the initial biases of the chosen superposition state.

In [ ]:
SingleSuperpositionPerturbation = quantumStateTransition(rulestxt, Tmax=4, nrshots=1000,
                                   initActivities=[90,90,135,90,90], initPerturbed=[False, False, True, False, False], thetatype="angle")
print("Single superposition perturbation of Pax6, biased towards OE:")
print(SingleSuperpositionPerturbation)

print("Double superposition perturbation of Pax6 and Coup_tfi, biased towards OE and KO respectively:")
DoubleSuperpositionPerturbation = quantumStateTransition(rulestxt, Tmax=4, nrshots=1000,
                                   initActivities=[90,90,135,90,45], initPerturbed=[False, False, True, False, True], thetatype="angle")
print(DoubleSuperpositionPerturbation)

### Using amplitude amplification to invert dynamics and identify predecessor states
This function call creates a circuit implementing multiple iterations $G$ of a Grover Oracle and Diffuser operator.
The marked attractor state 01101 is the solution element. The oracle operator includes the inverse circuit of the boolean state transition logic. The algorithm thus amplifies all predecessor states of the markedState (including this state itself if it is a fixed-point attractor).
That is, it effectly performs nrTransitions 'inverse' state transitions.
The amount by which the probability of the predecessor states is amplified depends on the number of iterations. This value does not increase monotonically with repeated iterations. The algorithm instead has a sinusoidal performance.
The optimal value for $G$ depends on the number of predecessor states, which can be determined via quantum counting as shown below.

In [ ]:
from qiskit_aer import Aer
from qiskit import execute
#Probability are rounded to 3 digits in resulting dictionaries

print("G=1 iteration:")
InvertedTransitionsCircuit_G1 = generate_groverSTGinversion_circuit(rulestxt, nrTransitions=2, markedState=[0,1,1,0,1], G=1)
result_G1 = execute(InvertedTransitionsCircuit_G1, backend=Aer.get_backend('qasm_simulator'), shots=1000).result()
countDict_G1 = outputformat(result_G1.get_counts(InvertedTransitionsCircuit_G1), normalizeOutput=True, sortOutput=True, digits=3)
print(countDict_G1)
solution_prob_G1 = countDict_G1["11001"] + countDict_G1["01001"] + countDict_G1["01101"] + countDict_G1["11101"]
print("After G=1 iteration, the M/N=4/32 solution states have a cumulative probability of " + str(solution_prob_G1) + ".")

print("G=2 iterations:")
InvertedTransitionsCircuit_G2 = generate_groverSTGinversion_circuit(rulestxt, nrTransitions=2, markedState=[0,1,1,0,1], G=2)
result_G2 = execute(InvertedTransitionsCircuit_G2, backend=Aer.get_backend('qasm_simulator'), shots=1000).result()
countDict_G2 = outputformat(result_G2.get_counts(InvertedTransitionsCircuit_G2), normalizeOutput=True, sortOutput=True, digits=3)
print(countDict_G2)
solution_prob_G2 = countDict_G2["11001"] + countDict_G2["01001"] + countDict_G2["01101"] + countDict_G2["11101"]
print("After G=2 iterations, the M/N=4/32 solution states have a cumulative probability of " + str(solution_prob_G2) + ".")

print("G=3 iterations:")
InvertedTransitionsCircuit_G3 = generate_groverSTGinversion_circuit(rulestxt, nrTransitions=2, markedState=[0,1,1,0,1], G=3)
result_G3 = execute(InvertedTransitionsCircuit_G3, backend=Aer.get_backend('qasm_simulator'), shots=1000).result()
countDict_G3 = outputformat(result_G3.get_counts(InvertedTransitionsCircuit_G3), normalizeOutput=True, sortOutput=True, digits=3)
print(countDict_G3)
solution_prob_G3 = countDict_G3["11001"] + countDict_G3["01001"] + countDict_G3["01101"] + countDict_G3["11101"]
print("After G=3 iterations, the M/N=4/32 solution states have a cumulative probability of " + str(solution_prob_G3) + ".")

### Quantum counting to estimate the number of solutions M
Performs a quantum counting algorithm, combining the Grover operator with a quantum Fourier transform on a separate register of readout qubits
of r_registerLen qubits. The bitstrings of resulting measurements now no longer indicate particular network states, but instead are integer
encodings for the total number $M$ of predecessors of the marked states.
E.g. the state 00000 has $M=14$ predecessors while the basin of the 01101 attractor has a size of $M=4$, requiring two inverted transitions.

In [ ]:
CountingResults_00000_Tinv1 = QuantumCountingAlgo(rulestxt, nrTransitions=1, markedState=[0,0,0,0,0],
                                                  r_registerLen=5, nrshots = 1000)

print(CountingResults_00000_Tinv1)

CountingResults_01101_Tinv2 = QuantumCountingAlgo(rulestxt, nrTransitions=2, markedState=[0,1,1,0,1],
                                                  r_registerLen=5, nrshots = 1000)

print(CountingResults_01101_Tinv2)

### Using IBM Cloud services and the IBM Runtime Sampler primitive to run circuits on the real IBMQ Algiers QPU
In order to run circuits on a real QPU, one must first register an account with IBM Cloud and set up a new instance with their quantum cloud service.
Thus, one will obtain an API key and a cloud resource number (CRN) as described in the protocol.

In [ ]:
apikey = "ENTER-YOUR-IBMCLOUD-API-KEY"
crn = "ENTER-YOUR-INSTANCES-CLOUD-RESOURCE-NUMBER"
name = "StarProtocol"

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session, Options
from qiskit import transpile
from qiskit.tools.visualization import plot_histogram

# Save an IBM Cloud account on disk and give it a name.
QiskitRuntimeService.save_account(channel="ibm_cloud", token=apikey, instance=crn, name=name)

In [ ]:
#Load the saved data to access runtime
service = QiskitRuntimeService(name="StarProtocol")
"""
By setting returnCircuitOnly=True, the quantumStateTransition() function return only the quantum circuit without performing a simulation on a local simulator as shown above.
The runtime service is used to access the ibm_algiers backend, available via the standard IBM Quantum cloud payment plan, along with the ibm_canberra device.
The circuit for performing a single state transition from a uniform superposition state is first transpiled to match the gate set of this backend and is then sent out to the QPU.
From here on out, the job is assigned a job_id and can be monitored in the 'Jobs' section of the corresponding instance in a browser.
The results are plotted, with states being decoded back to binary probabilities,
i.e. using "00011" instead of its integer correspondent "3" for a more clear evaluation of which genes are active or inactive.
"""
CorticalNet1Transition = quantumStateTransition(rulestxt, Tmax=1, returnCircuitOnly=True)
backend = service.get_backend('ibm_algiers')
CorticalNet1Transition_tr = transpile(CorticalNet1Transition, backend)
with Session(service=service, backend = backend):
    sampler = Sampler(options = Options(resilience_level=0))
    job = sampler.run(circuits=CorticalNet1Transition_tr, shots=100)
    result = job.result().quasi_dists[0].binary_probabilities()
    
print(result)
plot_histogram(result)

### Optional: extra error mitigation techniques
"trans_qc_list", created in the first line of the Multiple Transpilation code, contains a list of 20 different transpilations of the same algorithm. The following 3 lines of code select, among the 20 transpilations, the one containing the smallest number of CNOT gates, considered to be the most prone to noise among the basis gate of standard IBM quantum devices.
The last line defines "CorticalNet1Transition_tr", the official circuit to be executed, with the optimal transpilation spotted among the different options.
IMPORTANT: the more times the circuit is transpiled at the beginning ("times 20" in this example) the more optimization could be achieved, but deep circuits might take a long time to be transpiled so many times!

In [ ]:
service = QiskitRuntimeService(name=name) #Login to IBMQ runtime service
backend = service.get_backend('ibm_algiers') #Configure the quantum device to be used
CorticalNet1Transition = quantumStateTransition(rulestxt, Tmax=1, returnCircuitOnly=True) #Create the circuit simulating the Boolean network to be analyzed

# Here starts the Multiple Transpilation code
trans_qc_list = transpile([CorticalNet1Transition]*20, backend, optimization_level=3) #Generate 20 different transpilations for the circuit to be run
cx_count = [circ.count_ops()['cx'] for circ in trans_qc_list] #Count the number of CNOT gates contained in each different transpilation
best_idx = np.where(cx_count == np.min(cx_count))[0][0] #Pick the transpilation with the least amount of CNOT gates
best_qc = trans_qc_list[best_idx] #Set given transpilation to be the circuit to be run
CorticalNet1Transition_tr = transpile(best_qc,backend,optimization_level=0) #Create the transpiled circuit with the features evaluated above

#Run everything on the cloud platform
with Session(service=service, backend = backend):
    sampler = Sampler(options = Options(resilience_level=0))
    job = sampler.run(circuits=CorticalNet1Transition_tr, shots=100)
    result = job.result().quasi_dists[0].binary_probabilities()

print(result) #Retrieve and show the results
plot_histogram(result)

### Using AWS Braket to run circuits on an IonQ QPU
Similarly to the cloud service of IBM, accessing the IonQ device requires an account with AWS Braket. The setup of this account and the associated AWS buckets is explained in the protocol.
A state transition circuit is generated, which is manually transpiled to the set of basis gates available on the device.
As the result returned by the task includes measurements of all 11 qubits, the probabilities of states are summed up according to the values of the qubits of interests (representing the 5-gene bitstring output of the state transition). The bitstring notation is also converted between the Qiskit and Braket notations.

In [ ]:
from braket.aws import AwsDevice
nrshots = 100
cost = 0.3+0.01*nrshots #$0.30 / task + $0.01 / shot
print("NOTE: Running a task on the IonQ backend with this number of shots will create costs of $" + '{0:.2f}'.format(cost) + " USD!")

#Specify your account specific data here
my_bucket = "amazon-braket-YOUR-BUCKET-HERE"
my_prefix = "YOUR-FOLDER-NAME-HERE"
s3_folder = (my_bucket, my_prefix)
device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Harmony") #previously "arn:aws:braket:::device/qpu/ionq/ionQdevice", IonQ has now also added a second larger QPU named Aria

In [ ]:
from functionsForProtocol import *
import braket.circuits
from qiskit import *
from datetime import datetime
import boto3
from braket.devices import LocalSimulator
from braket.circuits import Circuit
import re #regex for parsing circuit to ionq

In [ ]:
GiacoCirc = synthesizeFullNetworkUpdateCircuit(rulestxt, update="synchronous")
n=5
InitCircuit = QuantumCircuit(2*n, n)
for q in range(5):
    InitCircuit.h(q)
GiacoCirc = InitCircuit.compose(GiacoCirc, list(range(0, 2 * n)), list(range(0, n)))

IonQCircuit = QiskitQASM2IonQCompiler(GiacoCirc, gatenameMapping=gatenameMapping_Qiskit_IonQ)

print("Circuit depth:")
print(IonQCircuit.depth) #Depth of 140 for single Giacomantonio transition circuit

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)
task = device.run(IonQCircuit, s3_folder, shots=nrshots, poll_timeout_seconds=24*60*60*7)  #7 day polling time before task times out
resultCounts = task.result().measurement_counts
print(resultCounts)
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

#IonQ measures all 11 qubits -> Parse output back into same format as was used in Qiskit simulations
qubitindices = [5,6,7,8,9] # Which qubits carry the outputs at t=1 for nodes in order, 0-indexed
resultCounts_qiskitFormat = ionqDict2qiskitNotation(resultCounts, qubitindices=qubitindices, invertEndian=True)
print(resultCounts_qiskitFormat)